In [11]:
import transformers

import datetime
import easydict
import itertools
import json
import matplotlib
import pathlib
import pprint
import re

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from operator import itemgetter
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from typing import List, Dict
import os

In [30]:
config = easydict.EasyDict({
    "raw_train_path": "./data/Training",
    "raw_valid_path": "./data/Validation",
    "clean": False,
    "data": "data", ## data path
    "seed": 42,
    "sample_submission_path": "sample_submission.tsv",
    "answer_path": "answer.tsv",
    "prediction_path": "prediction.tsv",
    "pretrained_model_name": "gogamza/kobart-base-v1",
    "train": "data/train.tsv",
    "valid": "data/valid.tsv",
    "test": "data/test.tsv",
    ## Training arguments.
    "ckpt": "ckpt", ## path
    "logs": "logs", ## path
    "per_replica_batch_size": 8,
    "gradient_accumulation_steps": 16,
    "lr": 5e-5,
    "weight_decay": 1e-2,
    "warmup_ratio": 0.2,
    "n_epochs": 10,
    "inp_max_len": 1024,
    "tar_max_len": 256,
    "model_fpath": "/content/drive/MyDrive/ColabNotebooks/dacon-gas/kobart-model.pth",
    ## Inference.
    "gpu_id": 0,
    "beam_size": 5,
    "length_penalty": 0.8,
    "no_repeat_ngram_size": 3,
    "var_len": False,
})

In [12]:
tokenizer = transformers.PreTrainedTokenizerFast.from_pretrained(
    'gogamza/kobart-base-v1'
)

tokenizer

PreTrainedTokenizerFast(name_or_path='gogamza/kobart-base-v1', vocab_size=30000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'mask_token': '<mask>'})

In [13]:
def print_elements(a: dict) -> None:
    pprint.PrettyPrinter(indent=4).pprint(a)

print_elements(vars(tokenizer))

{   '_additional_special_tokens': [],
    '_bos_token': '<s>',
    '_cls_token': None,
    '_decode_use_source_tokenizer': False,
    '_eos_token': '</s>',
    '_mask_token': '<mask>',
    '_pad_token': '<pad>',
    '_pad_token_type_id': 0,
    '_sep_token': None,
    '_tokenizer': <tokenizers.Tokenizer object at 0xc9d6b50>,
    '_unk_token': '<unk>',
    'deprecation_warnings': {},
    'init_inputs': (),
    'init_kwargs': {   'name_or_path': 'gogamza/kobart-base-v1',
                       'special_tokens_map_file': '/root/.cache/huggingface/transformers/3e6abf40f4fadbea9e7b539c182868d979838d8f7e6cdcdf2ed52ddcf01420c0.15447ae63ad4a2eba8bc7a5146360711dc32b315b4f1488b4806debf35315e9a'},
    'model_input_names': ['input_ids', 'token_type_ids', 'attention_mask'],
    'model_max_length': 1000000000000000019884624838656,
    'name_or_path': 'gogamza/kobart-base-v1',
    'padding_side': 'right',
    'verbose': True}


In [34]:
#file_list = os.listdir(folder_path)

def read_tsv(fpath: pathlib.PosixPath) -> pd.DataFrame:
    return pd.read_csv(
        fpath, 
        index_col=False,
        names=['summary', 'text'],
        sep="\t",
        encoding="utf-8")

In [35]:
import torch

class TextAbstractSummarizationDataset(torch.utils.data.Dataset):

    def __init__(
        self,
        tokenizer,
        fpath: pathlib.PosixPath,
        mode: str = "train",
    ):
        super(TextAbstractSummarizationDataset, self).__init__()

        self.df = read_tsv(fpath)
        # self.tok = tokenizer -> don't keep
        self.df.loc[:, "id"] = [i for i in range(self.df.shape[0])]
        
        ## Mode.
        assert mode in ["train", "test"]
        self.mode = mode

        ## Apply tokenize first to speed up in training phase and make code more simply.
        tqdm.pandas(desc="Tokenizing input texts")
        self.df.loc[:, "text_tok"] = self.df.loc[:, "text"].progress_apply(lambda x: tokenizer.encode(x))
        self.df.loc[:, "text_tok_len"] = self.df.loc[:, "text_tok"].apply(lambda x: len(x))

        if self.mode == "train":
            tqdm.pandas(desc="Tokenizing target summaries")
            self.df.loc[:, "summary_tok"] = self.df.loc[:, "summary"].progress_apply(lambda x: tokenizer.encode(x))
            self.df.loc[:, "summary_tok_len"] = self.df.loc[:, "summary_tok"].apply(lambda x: len(x))

        ## Sort by tokenized length with tqdm progress bar.
        ## 
        ## By sorting sequentially, starting with the longest sentence, 
        ## we can determine the maximum VRAM size the model is using for
        ## training. That is, if OOM does not occur for the maximum VRAM
        ## size at the beginning of training, it is guaranteed that OOM
        ## does not occur during training.
        self.df.sort_values(by=["text_tok_len"], axis=0, ascending=False, inplace=True)

    
    def __len__(self) -> int:
        return self.df.shape[0]


    def __getitem__(self, idx: int) -> Dict[str, List[int]]:
        instance = self.df.iloc[idx]

        return_value = {
            "id": instance["id"], ## for sorting in inference mode
            "text": instance["text_tok"],
            "length": len(instance["text_tok"]),
        }
        if self.mode == "train":
            return_value["summary"] = instance["summary_tok"]
        
        return return_value

In [36]:
class TextAbstractSummarizationCollator():

    def __init__(
        self,
        bos_token_id: int,
        eos_token_id: int,
        pad_token_id: int,
        inp_max_len: int = 1024,
        tar_max_len: int = 256,
        ignore_index: int = -100,
        mode: str = "train",
    ):
        super(TextAbstractSummarizationCollator, self).__init__()

        self.bos_token_id = bos_token_id
        self.eos_token_id = eos_token_id
        self.pad_token_id = pad_token_id
        self.inp_max_len = inp_max_len
        self.tar_max_len = tar_max_len
        self.ignore_index = ignore_index

        ## Mode.
        assert mode in ["train", "test"]
        self.mode = mode


    def _pad(self, sentences: List[List[int]], token_id: int) -> np.ndarray:
        ## We will pad as max length per batch, not "inp_max_len(=1024, etc)".
        max_length_per_batch = max([len(i) for i in sentences])

        ## Stack as dimension 0 (batch dimension).
        ## "token_id" can be "tokenizer.pad_token_id(=3)" or "ignore_index(=-100)"
        return np.stack([i + [token_id] * (max_length_per_batch - len(i)) for i in sentences], axis=0)


    def _train_collator(self, samples: List[Dict[str, List[int]]]) -> Dict[str, List[int]]:
        ## Unpack.

        ## If input max length > 1024, you can see below error:
        ##   1) Assertion `srcIndex < srcSelectDimSize` failed
        ##   2) Device-side assert triggered
        tokenized_texts     = [s["text"][:self.inp_max_len]        for s in samples]
        tokenized_summaries = [s["summary"][:self.tar_max_len - 1] for s in samples] ## <bos> or <eos> token index

        ## Inputs for encoder.
        input_ids = self._pad(tokenized_texts, token_id=self.pad_token_id)  ## numpy format
        attention_mask = (input_ids != self.pad_token_id).astype(float)     ## numpy format

        ## Inputs for decoder (generator).
        decoder_input_ids = [[self.bos_token_id] + i for i in tokenized_summaries]      ## bos
        decoder_input_ids = self._pad(decoder_input_ids, token_id=self.pad_token_id)    ## eos
        decoder_attention_mask = (decoder_input_ids != self.pad_token_id).astype(float)

        ## Answer.
        labels = [i + [self.eos_token_id] for i in tokenized_summaries]
        labels = self._pad(labels, token_id=self.ignore_index) ## why != "padding_id" ???

        ## We ensure that generator's inputs' and outputs' shapes are equal.
        assert decoder_input_ids.shape == labels.shape
        
        ## Pack as pre-defined arguments. See:
        ##   https://huggingface.co/docs/transformers/model_doc/bart#transformers.BartForConditionalGeneration
        return {
            "input_ids":                torch.from_numpy(input_ids),
            "attention_mask":           torch.from_numpy(attention_mask),
            "decoder_input_ids":        torch.from_numpy(decoder_input_ids),
            "decoder_attention_mask":   torch.from_numpy(decoder_attention_mask),
            "labels":                   torch.from_numpy(labels),
        }


    def _test_collator(self, samples: List[Dict[str, List[int]]]) -> Dict[str, List[int]]:
        ## Unpack.
        ids              = [s["id"]                      for s in samples]
        tokenized_texts  = [s["text"][:self.inp_max_len] for s in samples]   ## no <bos> token included

        ## Inputs for encoder.
        input_ids = self._pad(tokenized_texts, token_id=self.pad_token_id)  ## numpy format
        attention_mask = (input_ids != self.pad_token_id).astype(float)     ## numpy format

        ## Pack as pre-defined arguments:
        ## See: https://huggingface.co/docs/transformers/model_doc/bart#transformers.BartForConditionalGeneration
        return {
            "input_ids":        torch.from_numpy(input_ids),
            "attention_mask":   torch.from_numpy(attention_mask),
            ## Additional information to make answer.
            "id":               ids,
        }


    def __call__(self, samples: List[Dict[str, List[int]]]) -> Dict[str, List[int]]:
        return self._train_collator(samples) if self.mode == "train" else self._test_collator(samples)


def get_datasets(tokenizer, fpath: pathlib.PosixPath, mode: str = "train"):
    return TextAbstractSummarizationDataset(tokenizer, fpath, mode=mode)

In [37]:
tr_ds = get_datasets(
    tokenizer=tokenizer,
    fpath=Path("/workspace/home/uglee/Projects/title_extraction/datasets/integrated_pre_datasets/train_data.tsv")
)

vl_ds = get_datasets(
    tokenizer=tokenizer,
    fpath=Path("/workspace/home/uglee/Projects/title_extraction/datasets/integrated_pre_datasets/valid_data.tsv")
)

Tokenizing target summaries: 100%|██████████| 952/952 [00:00<00:00, 18471.04it/s]


In [38]:
len(tr_ds), len(vl_ds)

(952, 952)

In [39]:
model = transformers.BartForConditionalGeneration.from_pretrained(
    'gogamza/kobart-base-v1'
    )

loading configuration file https://huggingface.co/gogamza/kobart-base-v1/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/000a9f24bd41cd04e1263037d6b84bbc25b57d0f00a19f7540099c8a448cfeec.06f809b56ae6e8b04642a7106106b25f3e00faae652a6ee84df5fd19c5804ee2
Model config BartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.0,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2

In [40]:
## Path arguments.
nowtime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
output_dir = Path(config.ckpt, nowtime)
logging_dir = Path(config.logs, nowtime, "run")

training_args = transformers.Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    per_device_train_batch_size=config.per_replica_batch_size,
    per_device_eval_batch_size=config.per_replica_batch_size,
    gradient_accumulation_steps=config.gradient_accumulation_steps,
    learning_rate=config.lr,
    weight_decay=config.weight_decay,
    warmup_ratio=config.warmup_ratio,
    num_train_epochs=config.n_epochs,
    logging_dir=logging_dir,
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="epoch",
    # save_steps=1000,
    fp16=True,
    dataloader_num_workers=4,
    disable_tqdm=False,
    load_best_model_at_end=True,
    ## As below, only Seq2SeqTrainingArguments' arguments.
    # sortish_sampler=True,
    # predict_with_generate=True,
    # generation_max_length=config.tar_max_len,   ## 256
    # generation_num_beams=config.beam_size,      ## 5
)

## Print elements.
print_elements(vars(training_args))

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


{   '__cached__setup_devices': device(type='cuda', index=0),
    '_n_gpu': 4,
    'adafactor': False,
    'adam_beta1': 0.9,
    'adam_beta2': 0.999,
    'adam_epsilon': 1e-08,
    'dataloader_drop_last': False,
    'dataloader_num_workers': 4,
    'dataloader_pin_memory': True,
    'ddp_find_unused_parameters': None,
    'debug': [],
    'deepspeed': None,
    'disable_tqdm': False,
    'do_eval': True,
    'do_predict': False,
    'do_train': False,
    'eval_accumulation_steps': None,
    'eval_steps': None,
    'evaluation_strategy': <IntervalStrategy.EPOCH: 'epoch'>,
    'fp16': True,
    'fp16_backend': 'auto',
    'fp16_full_eval': False,
    'fp16_opt_level': 'O1',
    'generation_max_length': None,
    'generation_num_beams': None,
    'gradient_accumulation_steps': 16,
    'gradient_checkpointing': False,
    'greater_is_better': False,
    'group_by_length': False,
    'hub_model_id': None,
    'hub_strategy': <HubStrategy.EVERY_SAVE: 'every_save'>,
    'hub_token': None,
  

In [41]:
## Define trainer.
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=TextAbstractSummarizationCollator(
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        inp_max_len=config.inp_max_len,
        tar_max_len=config.tar_max_len,
    ),
    train_dataset=tr_ds,
    eval_dataset=vl_ds,
)

Using amp fp16 backend


In [42]:
## Just train.
trainer.train()

***** Running training *****
  Num examples = 952
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 512
  Gradient Accumulation steps = 16
  Total optimization steps = 10


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,No log,12.833954
1,No log,4.016085
2,No log,0.275976
3,No log,0.131680
4,No log,0.002076
5,No log,0.000611
6,No log,0.000331
7,No log,0.000170
8,No log,0.000059
9,4.753600,0.000034


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


***** Running Evaluation *****
  Num examples = 952
  Batch size = 32


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Saving model checkpoint to ckpt/20230116-013603/checkpoint-10
Configuration saved in ckpt/20230116-013603/checkpoint-10/config.json
Model weights saved in ckpt/20230116-013603/checkpoint-10/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ckpt/20230116-013603/checkpoint-10 (score: 3.35580189130269e-05).


TrainOutput(global_step=10, training_loss=4.753628540039062, metrics={'train_runtime': 270.6545, 'train_samples_per_second': 35.174, 'train_steps_per_second': 0.037, 'total_flos': 524444807577600.0, 'train_loss': 4.753628540039062, 'epoch': 9.53})